In [1]:
import tensorflow.keras as kr
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def path_to_array(path, img_size):
    
    """
    returns a (1, img_size, img_size, 3) matrix from values from 0 to 255
    """
    
    img_path = path
    img = kr.preprocessing.image.load_img(img_path, target_size=(img_size, img_size))
    x = kr.preprocessing.image.img_to_array(img)
    x = tf.expand_dims(x, 0)
    return x

In [3]:
def show_image(img):
    plt.figure()
    plt.imshow(img)

In [4]:
def predict(img, model):
    img = img / 255
    return model.predict(img)

In [ ]:
def intermediate_activation(img, model, layer, conv_net):
    l = model.get_layer(index = layer)
    layer_outputs = l.output[0, :, :, conv_net]
    print(layer_outputs)
    activation_model = kr.models.Model(inputs = model.input, outputs = layer_outputs)
    vis = activation_model.predict(img)
    return vis

In [ ]:
def feature_activation(model, layer, filter_no, step = 1, epoch=40):

    input_img_data = np.random.random((1, model.layers[0].input.shape[1], model.layers[0].input.shape[2], 3)) * 20 + 128
    input_img_data = tf.Variable(input_img_data, dtype="float32")
    acti_model = kr.models.Model(inputs = model.input, outputs = model.get_layer(index = layer).output[:, :, :, filter_no])
    for i in range(epoch):
        with tf.GradientTape(persistent=True) as gt:
            output= acti_model(input_img_data)
            loss = kr.backend.mean(output)
        
        gradients = gt.gradient(loss, input_img_data)        
        gradients /= (kr.backend.sqrt(kr.backend.mean(kr.backend.square(gradients))) + 1e-5)
        
        input_img_data.assign_add(gradients * step)
        
    input_img_data = input_img_data.numpy()    
    input_img_data -= input_img_data.mean()
    input_img_data /= (input_img_data.std() + 1e-5)
    input_img_data *= 0.1
    input_img_data += 0.5
    input_img_data = np.clip(input_img_data, 0, 1)
    input_img_data *= 255
    input_img_data = np.clip(input_img_data, 0, 255).astype('uint8')

    return input_img_data[0]